In [ ]:
conda install -c conda-forge lightgbm

In [ ]:
conda install -c conda-forge xgboost

In [ ]:
conda install -c conda-forge catboost

In [2]:
# Import packages
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Regression algorithms
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [3]:
# Load data
df = pd.read_csv('全国.csv', encoding='gbk')

# Convert the relevant columns to categorical data type
df['Number'] = df['Number'].astype('category')
df['Season'] = df['Season'].astype('category')
df['Treating_Process'] = df['Treating_Process'].astype('category')
df['Phosphorus_Removing_Agents'] = df['Phosphorus_Removing_Agents'].astype('category')
df['Carbon_Source'] = df['Carbon_Source'].astype('category')
df['City'] = df['City'].astype('category')

# Get the column names
column_names = df.columns

In [4]:
# Fit the model
# Variables to use as X
Names_X = [
    'CODin', 'CODrem',
    'BODin/CODin', 'BODrem',
    'NH3-Nin', 'NH3-Nrem',
    'SSin', 'SSrem',
    'TNin', 'TNrem',
    'TPin', 'TPrem',
    'Season',
    'MonthVolume',
    'Carbon_Source',
    'Mass_Carbon_Source(kg/m3)',
    'Phosphorus_Removing_Agents',
    'Mass_Phosphorus_Removing_Agents(kg/m3)',
    'Treating_Process',
    'City',
]
# Variable to use as y
Names_y = 'DS(kg/m3)'

# Define X and y
X = df[Names_X]
y = df[Names_y]

In [5]:
# Create an instance of GroupShuffleSplit, spliting the data
# Such that the training and test sets have different plants
gss = GroupShuffleSplit(n_splits=5, test_size=0.15, random_state=7)

# Split the data
train_indices, test_indices = next(gss.split(X, y, groups=df['Number']))
# Create the training and testing sets
X_train, y_train = X.iloc[train_indices], y.iloc[train_indices]
X_test, y_test = X.iloc[test_indices], y.iloc[test_indices]

# The whole dataframe corresponding to the training set
df_train = df.iloc[train_indices]

In [6]:
pipe = Pipeline(steps=[
   ('rgr', GradientBoostingRegressor())])

# Set the options for estimators and hyperparemeters
rgr_list = [
    {'rgr': [LinearRegression()]},  # Linear Regression
    {'rgr': [Lasso()],
     'rgr__alpha': [0.1, 1, 10]},  # LASSO
    {'rgr': [KNeighborsRegressor()],
     'rgr__n_neighbors': [5, 6, 7]},  # K-nearest Neighbors
    {'rgr': [SVR()],
     'rgr__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
     'rgr__C': [0.1, 1, 10]},  # Support Vector Machines
    {'rgr': [RandomForestRegressor(n_estimators=400)],
     'rgr__max_depth': [1, 2, 5],
     'rgr__max_features': ['sqrt','log2',None],
     'rgr__min_samples_split': [2,3,4,5,6,7,8,9],
     'rgr__min_impurity_decrease': [0,0.05,0.1],
     'rgr__bootstrap': [True,False]},  # Random Forests
    {'rgr': [MLPRegressor(random_state=1)],
     'rgr__activation': ['logistic', 'tanh', 'relu'],
     'rgr__hidden_layer_sizes': [(100,), (15,)],
     'rgr__alpha': [1e-4, 1e-5]},  # Neural Networks
    {'rgr': [GradientBoostingRegressor(),
              LGBMRegressor(), XGBRegressor(),
              CatBoostRegressor(silent=True)],
     'rgr__n_estimators': [100, 200, 500],
     'rgr__max_depth': [3,4,5,6,9],
     'rgr__learning_rate': [0.01, 0.05, 0.1, 0.2]},  # Gradient Boosting Machines
    {'rgr': [LGBMRegressor()],
     'rgr__n_estimators': [100, 200, 500],
     'rgr__max_depth': [3,4,5,6,9],
     'rgr__reg_lambda': [0,0.1,1,2,3],
     'rgr__learning_rate': [0.01, 0.05, 0.1, 0.2]}  # LightGBM
]

# Combine options for features and options for estimators
param_grid = [{**rgr} for rgr in rgr_list]

# Search
grid_search = GridSearchCV(pipe, param_grid=param_grid,
                           cv=gss.split(X_train, y_train, groups=df_train['Number']),
                           scoring='r2',n_jobs=-1)

In [8]:
from sklearn.metrics import r2_score, mean_absolute_error

# Fit the model
grid_search.fit(X_train, y_train)

# Calculate R2 score of the training set
R2_train_BestModel = grid_search.score(X_train, y_train)

# Print the Training set R2
print('\nTraining set R²:', R2_train_BestModel)

# Print the best hyperparameters and the corresponding score
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)
print('Sample Size:', len(y_train))

# Prediction on the test set
y_pred_BestModel = grid_search.predict(X_test)
# R2 on the test set
R2_Test_BestModel = r2_score(y_test, y_pred_BestModel)
# Mean Absolute Error (MAE) on the test set
MAE_Test_BestModel = mean_absolute_error(y_test, y_pred_BestModel)
# Normalized Mean Absolute Error (NMAE)  on the test set
NMAE_Test_BestModel = MAE_Test_BestModel / y_test.mean() * 100

# Print results
print('\nTest set R²:', R2_Test_BestModel)
print('Test set Mean Absolute Error:', MAE_Test_BestModel)
print('Test set Normalized Mean Absolute Error:',
      NMAE_Test_BestModel, 'percent')
print('Sample Size:', len(y_pred_BestModel))

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
600 fits failed out of a total of 5015.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._final_estim


Training set R²: 0.3683617393752232
Best hyperparameters: {'rgr': GradientBoostingRegressor(learning_rate=0.01, n_estimators=500), 'rgr__learning_rate': 0.01, 'rgr__max_depth': 3, 'rgr__n_estimators': 500}
Best score: 0.16696642500501574
Sample Size: 5831

Test set R²: 0.29744051496437607
Test set Mean Absolute Error: 0.03930696320361723
Test set Normalized Mean Absolute Error: 34.86434496098318 percent
Sample Size: 826


In [9]:
# Test in Key Basins
KeyCity = [240]
mask = X_test['City'].isin(KeyCity)
X_test_KeyCity = X_test[mask]
y_test_KeyCity = y_test[mask]

# Prediction on the KeyCity
y_pred_BestModel_KeyCity = grid_search.predict(X_test_KeyCity)
# R2 on the test set
R2_Test_BestModel_KeyCity = r2_score(y_test_KeyCity, y_pred_BestModel_KeyCity)
# Mean Absolute Error (MAE) on the test set
MAE_Test_BestModel_KeyCity = mean_absolute_error(y_test_KeyCity, y_pred_BestModel_KeyCity)
# Normalized Mean Absolute Error (NMAE) on the test set
NMAE_Test_BestModel_KeyCity = MAE_Test_BestModel_KeyCity / y_test_KeyCity.mean() * 100
# Print results
print('\nTest set R² in KeyCity:',
      R2_Test_BestModel_KeyCity)
print('Test set Mean Absolute Error in KeyCity:',
      MAE_Test_BestModel_KeyCity)
print('Test set Normalized Mean Absolute Error in KeyCity:',
      NMAE_Test_BestModel_KeyCity, 'percent')
print('Sample Size:', len(y_pred_BestModel_KeyCity))


Test set R² in KeyCity: 0.13369312671853462
Test set Mean Absolute Error in KeyCity: 0.041777016723734295
Test set Normalized Mean Absolute Error in KeyCity: 33.59154430236288 percent
Sample Size: 70


In [6]:
from sklearn.model_selection import GroupShuffleSplit
help (GroupShuffleSplit)

Help on class GroupShuffleSplit in module sklearn.model_selection._split:

class GroupShuffleSplit(ShuffleSplit)
 |  GroupShuffleSplit(n_splits=5, *, test_size=None, train_size=None, random_state=None)
 |  
 |  Shuffle-Group(s)-Out cross-validation iterator
 |  
 |  Provides randomized train/test indices to split data according to a
 |  third-party provided group. This group information can be used to encode
 |  arbitrary domain specific stratifications of the samples as integers.
 |  
 |  For instance the groups could be the year of collection of the samples
 |  and thus allow for cross-validation against time-based splits.
 |  
 |  The difference between LeavePGroupsOut and GroupShuffleSplit is that
 |  the former generates splits using all subsets of size ``p`` unique groups,
 |  whereas GroupShuffleSplit generates a user-determined number of random
 |  test splits, each with a user-determined fraction of unique groups.
 |  
 |  For example, a less computationally intensive alternat